In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.metrics import roc_curve, auc
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
from tqdm import tqdm_notebook as tqdm
sns.set(rc={'figure.figsize':(15.7,8.27)})

In [2]:
n_rep = 100
n_bins = 1000
n_nodes = 10
n_regions = 40
n_reads = 10000
PATH = '../../workflows/bp_detection_performance/results/'

In [3]:
gt_files = [PATH + str(n_nodes)+'nodes_'+str(n_regions)+'regions_'+str(n_reads) \
             +'reads_simdata_'+str(x)+'_ground_truth.txt' for x in range(0,n_rep)]
bps_files = [PATH + 'simdata_'+str(x)+'_all_bps_comparison.csv' for x in range(0,n_rep)]
all_bins = range(0,n_bins)

In [ ]:
all_bins

range(0, 1000)

In [ ]:
all_tpr = []
all_fpr = []
all_n_positives = []
all_bps_tables = []

threshold_coeffs = np.linspace(1.0,16.0, 100) # 100 thresholds btw 1 and 16

for gt_file, bps_file in tqdm(zip(gt_files, bps_files)):
    bps = pd.read_csv(bps_file, header=None)
    bps.columns = ['idx','log_sp','stdev']
    bps['ranking'] = bps['log_sp'] / bps['stdev']
    # bps = bps.sort_values('ranking',ascending=False)
    bps = bps.dropna()
    
    all_bps_tables.append(bps)
    
    # get the ground truth
    cell_genotypes = pd.read_csv(gt_file, sep=' ' ,header=None)
    cell_genotypes = cell_genotypes[cell_genotypes.columns[:-1]] # remove the last (only NaN) column
    cell_bps = cell_genotypes.diff(periods=1, axis=1) # apply diff to detect breakpoints
    cell_bps = cell_bps.fillna(value=0.0) # diff makes the 1st row NaN, make it zero
    cell_bps[cell_bps != 0] = 1 # replace the non-zeroes by 1
    grouped_cell_bps = cell_bps.sum(axis=0) # count the non-zeroes
    ground_truth = grouped_cell_bps[grouped_cell_bps > 0] # if they occur in at least 1 cell
    ground_truth = ground_truth.index.tolist()
    # end of ground truth    

    # correcting for the bps 1-2 bins nearby
    for index, row in bps.iterrows():
        idx_val = bps.loc[index, 'idx']
        for gt in ground_truth:
            if (abs(idx_val - gt) <=2 and idx_val != gt):
                print('correcting ' + str(idx_val) + '->' + str(gt))
                bps.loc[index,'idx'] = gt
    
    # use a fixed threshold_coeffs for all
    # threshold_coeffs = sorted(bps['ranking'].values)
    print('len threshold coeffs' + str(len(threshold_coeffs)))
    # Each breakpoint candidate has a different stdev value.
    # The ROC computations takes that into account.tpr_values = []
    tpr_values = []
    fpr_values = []
    n_positives = []
    for thr in threshold_coeffs:
        predicted_positives = []
        predicted_negatives = []
        for index, row in bps.iterrows():
            if row['ranking'] > thr:
                predicted_positives.append(row['idx'])
            else:
                break 
                
        #import ipdb; ipdb.set_trace()
        predicted_negatives = [i for i in all_bins if i not in predicted_positives]

        true_positives = [i for i in predicted_positives if i in ground_truth]
        false_positives = [i for i in predicted_positives if i not in ground_truth]

        true_negatives = [i for i in predicted_negatives if i not in ground_truth]
        false_negatives = [i for i in predicted_negatives if i in ground_truth]

        # import ipdb; ipdb.set_trace()
        assert(len(ground_truth) == (len(true_positives) + len(false_negatives)))
        tpr = len(true_positives) / len(ground_truth) # len(ground_truth)
        fpr = len(false_positives) / (1000 - len(ground_truth)) # (len(false_positives) + len(true_negatives))
        tpr_values.append(tpr)
        fpr_values.append(fpr)
        n_positives.append(len(predicted_positives))
    
    all_tpr.append(tpr_values)
    all_fpr.append(fpr_values)
    all_n_positives.append(n_positives)

A Jupyter Widget

correcting 652->651
correcting 206->207
correcting 526->525
correcting 762->761
correcting 881->880
len threshold coeffs100
correcting 366->365
len threshold coeffs100
correcting 644->645
correcting 888->889
correcting 598->597
len threshold coeffs100
correcting 674->673
correcting 803->804
len threshold coeffs100
correcting 628->629
correcting 446->447
correcting 206->207
len threshold coeffs100
correcting 441->440
correcting 912->913
correcting 871->869
correcting 198->199
len threshold coeffs100
correcting 926->927
len threshold coeffs100
correcting 390->391
len threshold coeffs100
correcting 187->189
correcting 138->137
correcting 648->647
len threshold coeffs100
correcting 485->484
correcting 504->505
correcting 521->520
len threshold coeffs100
correcting 577->579
len threshold coeffs100


In [ ]:
auc_vals = []
plt.figure(figsize=(8,8))
for tpr_values, fpr_values in zip(all_tpr, all_fpr):
    roc_auc = auc(fpr_values, tpr_values)
    auc_vals.append(roc_auc)
    plt.plot(fpr_values, tpr_values) # label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.gca().set_aspect('equal', adjustable='box')
plt.grid(True)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
# plt.legend(loc="lower right")
plt.show()

In [ ]:
threshold_indices = []
for idx, val in enumerate(all_tpr):
    df = pd.DataFrame(data=[all_tpr[idx],all_fpr[idx]]).T
    df.columns = ['tpr','fpr']
    df['idx'] = df.index
    df = df[df['tpr']==df['tpr'].max()]
    df = df[df['fpr']==df['fpr'].min()]
    print(df)
    thr_idx = df.idx.values[0]
    threshold_indices.append(thr_idx)

In [ ]:
len(threshold_indices)

In [ ]:
len(all_tpr[0])

In [ ]:
all_bps_tables[0].shape

In [ ]:
all_bps_tables[0]

In [ ]:
bps_thresholds = []
for idx, val in enumerate(threshold_indices):
    bps_thresholds.append(all_bps_tables[idx].iloc[[val]]['ranking'].values[0])

In [ ]:
sns.set(rc={'figure.figsize':(15.7,8.27)})
ax = sns.distplot(bps_thresholds, rug=True);
ax.xaxis.set_major_locator(ticker.MultipleLocator(0.5))

## Average TRP, FPR, number of predicted positives for each threshold value over 100 runs
For each run.

In [ ]:
tpr_df = pd.DataFrame(all_tpr)
tpr_df.columns = threshold_coeffs
tpr_df.head()

In [ ]:
mean_tpr = tpr_df.mean()
plt = sns.lineplot(x=mean_tpr.index,y=mean_tpr.values).set_title('Average TPR values')

In [ ]:
fpr_df = pd.DataFrame(all_fpr)
fpr_df.columns = threshold_coeffs
fpr_df.head()

In [ ]:
mean_fpr = fpr_df.mean()
plt = sns.lineplot(x=mean_fpr.index,y=mean_fpr.values).set_title('Average FPR values')

In [ ]:
n_positives_df = pd.DataFrame(all_n_positives)
n_positives_df.columns = threshold_coeffs
n_positives_df.head()

In [ ]:
mean_n_positives = n_positives_df.mean()
plt = sns.lineplot(x=mean_n_positives.index,y=mean_n_positives.values).set_title('Average n_positive values')

In [ ]:
df_avg = pd.DataFrame(data={'mean_tpr': mean_tpr, 'mean_fpr': mean_fpr})

In [ ]:
df_avg.head()

In [ ]:
df_avg.plot()